In [17]:
# imports

import datetime
import searchtweets
import pandas as pd

import string
import warnings
import datetime
warnings.filterwarnings('ignore')
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import seaborn as sns
from IPython.display import display
from wordcloud import WordCloud, STOPWORDS

[nltk_data] Downloading package stopwords to /home/keras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
curated_list = ['AwkwardRambler','AngryBlackN8V',
             'Pandabbadon', 'LammaticHama','DeadDogLake',
             'cricketcrocker','devilishgrin000','LowArctic',
             'ChelseyMooner','akayatuk','nativeopinion',
             'BadSalishHeart','joyem_braun','RuthH_Hopkins',
             'brettachapman','NativeApprops','Pam_Palmater',
             'AzieDee','BigIndianGyasi','RoanhorseBex',
             'Leave_Matoaka','ZoeSTodd',
             'notvanishing','Dallas_Hunt','justicedanielh',
             'WordsandGuitar','tuckeve','tagaq',
             'TanyaTalaga','Hayden_King','gindaanis',
             'antalalakam','dearnonnatives','VinceSchilling',
             'DelSchilling','LowaBeebe',
             'llewellynjobs','DaveAlexRoberts','SaltyLilOjibwe',
             'DarrylLeroux','mredshirtshaw','mors_lakota',
             'Ruth4Nd','Indigenia','ItazipcoLakota',
             'ryanredcorn','KimTallBear','tanayawinder',
             'shotheekwe','msimmons444','powhatannative',
             'aliwatson117','jfkeeler','ThunderingElks',
             'FrankWaln','AgentNdn','DeLesslin']

In [6]:
print(len(curated_list))

57


In [27]:
# @justicedanielh, @KimTallBear on curated list also

I_scholars_list = ['justicedanielh', 'PepePierce', 'pollysgdaughter', 'rebeccanagle', 'KimTallBear']

In [ ]:
# need to get historical tweets from curated list & scholars
# might as well do it all at once

full_list = ['AwkwardRambler','AngryBlackN8V',
             'Pandabbadon', 'LammaticHama','DeadDogLake',
             'cricketcrocker','devilishgrin000','LowArctic',
             'ChelseyMooner','akayatuk','nativeopinion',
             'BadSalishHeart','joyem_braun','RuthH_Hopkins',
             'brettachapman','NativeApprops','Pam_Palmater',
             'AzieDee','BigIndianGyasi','RoanhorseBex',
             'Leave_Matoaka','ZoeSTodd',
             'notvanishing','Dallas_Hunt','justicedanielh',
             'WordsandGuitar','tuckeve','tagaq',
             'TanyaTalaga','Hayden_King','gindaanis',
             'antalalakam','dearnonnatives','VinceSchilling',
             'DelSchilling','LowaBeebe',
             'llewellynjobs','DaveAlexRoberts','SaltyLilOjibwe',
             'DarrylLeroux','mredshirtshaw','mors_lakota',
             'Ruth4Nd','Indigenia','ItazipcoLakota',
             'ryanredcorn','KimTallBear','tanayawinder',
             'shotheekwe','msimmons444','powhatannative',
             'aliwatson117','jfkeeler','ThunderingElks',
             'FrankWaln','AgentNdn','DeLesslin',
             'PepePierce','pollysgdaughter','rebeccanagle']

In [34]:
def get_tweet_object(search_term, start_date, end_date, num_of_tweets, include_retweets=True):
    
    '''
    takes date range, screen name & number of tweets
    
    search returns replies to input screen name
    
    returns tweets object
    
    input search term as a string
    
    input search range as datetime.datetime(20xx, 1, 20)
    
    include_retweets defaults is True. to exclude retweets set to False.
    
    '''
    
    
    search_args = searchtweets.load_credentials('twitter_keys.yaml',
                                                yaml_key='search_tweets_api',
                                                env_overwrite=False)
    
    if include_retweets:
        
        #search_term = search_term + ' lang:en'
        
        search_term = '(to:' + search_term + ') is:reply lang:en'
    
    else:
        
        #search_term = search_term + ' lang:en -is:retweet'
        
        search_term = '(to:' + search_term + ') is:reply lang:en -is:retweet'
        
    
    
    rule = searchtweets.gen_rule_payload(search_term,
                                         results_per_call=500,
                                         from_date=start_date.strftime('%Y-%m-%d'),
                                         to_date=end_date.strftime('%Y-%m-%d'))     
        
    tweet_objects = []
    
    rs = searchtweets.ResultStream(rule_payload=rule,
                                   max_results=num_of_tweets,
                                   max_pages= num_of_tweets / 500,
                                   **search_args)
    
    start_time = time.time()

    for i, tweet_object in enumerate(rs.stream(), start=1):
        
        if i % 500 == 0:
            
            now = time.time()
            
            if now < start_time + 4:
                
                time.sleep(4 - (now - start_time))
                
                start_time = time.time()
                
        tweet_objects.append(tweet_object)
    
    return tweet_objects

def tweet_obj_to_df(tweet_objects):
    
    tweet_dict = {'text': [],
                  'author': [],
                  'replying_to': []
                 }
    
    for tweet in tweet_objects:
        
        tweet_dict['text'].append(tweet.all_text)
        
        tweet_dict['author'].append(tweet.screen_name)
        
        tweet_dict['replying_to'].append(tweet.in_reply_to_screen_name)
    
    tweets_df = pd.DataFrame(tweet_dict)
    
    return tweets_df

In [33]:
# get_tweet_object(search_term, start_date, end_date, num_of_tweets, include_retweets=True):

# search_term = '(to:USERNAME) is:reply'

test_tweet_object = get_tweet_object('rebeccanagle', datetime.datetime(2018, 1, 1), datetime.datetime(2020, 1, 26), 100, include_retweets=False)

test_tweets_df = tweet_obj_to_df(test_tweet_object)

print(len(test_tweets_df), '\n')

print(test_tweets_df.head(20))
    

Grabbing bearer token from OAUTH


100 

                                                 text           author  \
0   @rebeccanagle Sound like a Bernie Slogan... #N...  RichardFerreiro   
1                              @rebeccanagle Truth!!!      bear_daddys   
2                   @rebeccanagle That sounds “great”         jbecks74   
3   @rebeccanagle @KimTallBear @highcountrynews I ...        Jamie_Maz   
4   @rebeccanagle is there anyway you remember the...     nrtankersley   
5   @rebeccanagle @peterdaou The same publication ...  A_Savage_Indian   
6   @rebeccanagle @DianeSnavely @KimTallBear @high...   TammyKosiancic   
7   @rebeccanagle @DianeSnavely @KimTallBear @high...   TammyKosiancic   
8   @rebeccanagle Nishnabè which sort of translate...       sour_utley   
9   @rebeccanagle Kanieknehake, means People of th...  wrdswthstranger   
10  @rebeccanagle @peterdaou I knew nothing about ...       ObserverPa   
11  @rebeccanagle @highcountrynews @KimTallBear A ...           TomLeb   
12  @rebeccanagle @highcountryne

In [14]:
#search_term = '(to:USERNAME) is:reply'

searchy_term = 'username'

print('(to:',searchy_term,') is:reply')

(to: username ) is:reply
